In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import dep.arcturis as arc
import gpt as kadabra

from openai import OpenAI

In [2]:
#initializaion, login and problem select
client = OpenAI()

language = 'Python'

options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
driver.get("https://leetcode.com/accounts/login/")


def wait_and_find_element(driver, by, value):
    return WebDriverWait(driver, 10).until(EC.presence_of_element_located((by, value)))

login_field = wait_and_find_element(driver, By.ID, "id_login")
login_field.send_keys(arc.e)    

password_field = wait_and_find_element(driver, By.ID, "id_password")
password_field.send_keys(arc.p)

EC.element_to_be_clickable((By.XPATH, "//div[@class='btn-content-container__2HVS']/span[text()='Sign In']"))
WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, "initial-loading")))

# Find the "Sign In" button
sign_in_button = wait_and_find_element(driver,By.XPATH, "//div[@class='btn-content-container__2HVS']/span[text()='Sign In']")

driver.execute_script("arguments[0].scrollIntoView(true);", sign_in_button)

time.sleep(5)
sign_in_button.click()
time.sleep(3)


problems = wait_and_find_element(driver, By.XPATH,"//a[@href='/problemset/all/']")
problems.click()

pickone = wait_and_find_element(driver, By.XPATH,"//div[contains(@class, 'ml-auto')]")
# Click the button  
pickone.click()

In [3]:
#pophandler loop
def click_next_button(driver):
    #find all of the elements in the navbar with "cursor-pointer" and click the 3rd one (next problem)
    navbar = wait_and_find_element(driver, By.CLASS_NAME,"z-nav-1")
    buttons = navbar.find_elements(By.CLASS_NAME,"cursor-pointer")
    buttons[2].click()  

def handle_subscribe_popup(driver):
    while True:
        try:
            # Check for the subscribe popup
            subscribe_popup = WebDriverWait(driver, 4).until(                
                EC.visibility_of_element_located((By.XPATH, "//div[contains(text(), 'Subscribe to unlock.')]"))
            )

            click_next_button(driver)
                

        except:
            # If the subscribe popup is not found, break out of the loop
            break

  

#need to check language and continue looping through pop up check
handle_subscribe_popup(driver)


In [90]:
#language checker
"""
4.9.24 - need to check the language, select the language I want
 try working in python first in case of debug or formatting issues
"""


# Find the button element within the div element
button = wait_and_find_element(driver, By.CSS_SELECTOR, ".rounded.items-center.whitespace-nowrap.focus\\:outline-none.inline-flex.bg-transparent.dark\\:bg-dark-transparent.text-text-secondary.dark\\:text-text-secondary.active\\:bg-transparent.dark\\:active\\:bg-dark-transparent.hover\\:bg-fill-secondary.dark\\:hover\\:bg-fill-secondary.px-1\\.5.py-0\\.5.text-sm.font-normal.group")

# Print the text of the button
print(button.text)

time.sleep(1)
button.click()

element = wait_and_find_element(driver, By.XPATH, "//div[contains(text(), 'C++')]")
time.sleep(1)
# Click the element
element.click()


C++


In [94]:
#this works to pick the python button
button = wait_and_find_element(driver, By.CSS_SELECTOR, ".rounded.items-center.whitespace-nowrap.focus\\:outline-none.inline-flex.bg-transparent.dark\\:bg-dark-transparent.text-text-secondary.dark\\:text-text-secondary.active\\:bg-transparent.dark\\:active\\:bg-dark-transparent.hover\\:bg-fill-secondary.dark\\:hover\\:bg-fill-secondary.px-1\\.5.py-0\\.5.text-sm.font-normal.group")
button.click()
element = wait_and_find_element(driver, By.XPATH, "//div[@class='group flex min-w-[140px] cursor-pointer items-center justify-between rounded-[4px] py-[6px] pl-[6px] pr-3 hover:bg-fill-tertiary dark:hover:bg-fill-tertiary']//div[contains(text(), 'Python')]")
element.click()


In [ ]:
language = "Python"

def solution_ai(driver,language):
    time.sleep(3)
    #extract the text in the problem, send it into the gpt prompt, clear the field, input sol 

    description_element = wait_and_find_element(driver,By.CLASS_NAME,'elfjS')
    description_text = description_element.text

    code_block_element = wait_and_find_element(driver,By.CSS_SELECTOR,'.view-lines')
   
    code_block_text = code_block_element.text

    line_elements = code_block_element.find_elements(By.CLASS_NAME, 'view-line')

    sol = kadabra.main(f"I would like you to solve a problem in {language}, please only return code, do not inlcude any additional text and do not add comments within the code, do not use use any shortcuts for brevity (write the entire solution) and do not add ```cpp or ``` at the beginning or end of the solution. Here is the description of the problem: {description_text} and use this format to provide the solution {code_block_text}")

    container = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    #clear cells
    driver.execute_script("arguments[0].innerText = '';", container)
    block_of_text = sol
    #use javascript to import solutions
    driver.execute_script("arguments[0].innerText = arguments[1];", container, block_of_text)
    print(sol)

solution_ai(driver,language)



In [6]:

language = "C++"
sol = ''
def solution_ai2(driver, language):
    time.sleep(3)
    global sol
    
    # Extract the text in the problem
    description_element = wait_and_find_element(driver, By.CLASS_NAME, 'elfjS')
    description_text = description_element.text

    code_block_element = wait_and_find_element(driver, By.CSS_SELECTOR, '.view-lines')
    code_block_text = code_block_element.text    

    sol = kadabra.main(f"I would like you to solve a problem in {language}, please only return code, do not include any additional text and do not add comments within the code, do not use any shortcuts for brevity (write the entire solution) and do not add ```cpp or ``` at the beginning or end of the solution. Here is the description of the problem: {description_text} and use this format to provide the solution {code_block_text}")
    num_lines = len(sol.split('\n'))

    container = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    
    # Clear cells
    driver.execute_script("arguments[0].innerText = '';", container)
    
    # Add newline characters before inserting the block of text
    block_of_text = '\n'.join(sol.split('\n'))
    
    # Use JavaScript to import solutions
    driver.execute_script("arguments[0].innerText = arguments[1];", container, block_of_text)
    
    print(f"Number of lines in solution: {num_lines}")
    print(sol)

solution_ai2(driver, language)


Number of lines in solution: 25
#include <vector>

using namespace std;

class Solution {
public:
    int findDuplicate(vector<int>& nums) {
        int slow = nums[0];
        int fast = nums[0];
        
        do {
            slow = nums[slow];
            fast = nums[nums[fast]];
        }while (slow != fast);

        slow = nums[0];

        while (slow != fast) {
            slow = nums[slow];
            fast = nums[fast];
        }

        return fast;
    }
};


In [141]:
print(sol.count('}'))

4


In [77]:
#This works to press enter and backspace, particularly on a fresh instance
time.sleep(3)

length = len(sol.split('\n'))

clicktry = wait_and_find_element(driver, By.XPATH,"//div[@class='view-line']")

ActionChains(driver).move_to_element(clicktry).send_keys(Keys.ENTER).perform()

ActionChains(driver).move_to_element(clicktry).send_keys(Keys.BACKSPACE).perform()





In [ ]:
#try pressing enter on element
time.sleep(4)
container = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
element = container.find_elements(By.CLASS_NAME, 'view-line')

for num in range(20):
    driver.execute_script("arguments[0].dispatchEvent(new KeyboardEvent('keydown', {key: 'Enter'}));", container)

    from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# Find the element you want to interact with
element = driver.find_element_by_xpath("//div[@class='view-line']")

# Perform the key press action to simulate pressing Enter
ActionChains(driver).move_to_element(element).send_keys(Keys.ENTER).perform()


In [87]:
#I may need to find new selectors for the code editor field

#I want to have a script to count how many line elements exist

from selenium import webdriver
from selenium.webdriver.common.by import By


container = driver.find_element(By.CLASS_NAME, "view-lines")

# Find all elements with the class name 'view-line' within the container
view_lines = container.find_elements(By.CLASS_NAME, "view-line")

driver.execute_script("arguments[0].scrollIntoView(false);", container)

# Count the number of 'view-line' elements
num_view_lines = len(view_lines)
# Output the count
print("Number of view-line elements:", num_view_lines)



Number of view-line elements: 9


In [122]:
#works to press enter, backspace, up, down, and sending text with an actionchains


time.sleep(3)

container2 = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")


ActionChains(driver).move_to_element(container2).send_keys(Keys.ARROW_UP).perform()

time.sleep(1)



for _ in range(3):
    time.sleep(1)
    container3 = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    ActionChains(driver).move_to_element(container3).send_keys(Keys.ARROW_UP).perform()
    print('loop arrows')




element = wait_and_find_element(driver, By.XPATH, "//div[@class='view-line']")

ActionChains(driver).move_to_element(container2).send_keys("test").perform()




loop arrows
loop arrows
loop arrows


In [137]:
#need to use my new skills to insert the solution properly 

litch = sol.split('\n')

textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
element = wait_and_find_element(driver, By.XPATH, "//div[@class='view-line']")

for line in litch:
    ActionChains(driver).move_to_element(textwindow).send_keys(line).perform()
    ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ENTER).perform()

#press format code

formatcode = driver.find_element_by_xpath("//svg[@data-icon='align-left']")

# Click the SVG button
formatcode.click()




In [135]:

time.sleep(2)

#this clears the text

code_block_element = wait_and_find_element(driver, By.CSS_SELECTOR, '.view-lines')
charstoclear = len(code_block_element.text)

textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")

textwindow.click()
# for _ in range(30):
#     textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
#     ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ARROW_DOWN).perform()

for _ in range(charstoclear+charstoclear):
    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    ActionChains(driver).move_to_element(textwindow).send_keys(Keys.BACKSPACE).perform()


In [5]:
#official


solution = ''
def solution_input(driver, language):
    #extracts the description and pre-formatted code, inserts into a gpt prompt
    #clears the text window and inserts the solution

    global solution
    
    description_element = wait_and_find_element(driver, By.CLASS_NAME, 'elfjS')
    description_text = description_element.text

    code_block_element = wait_and_find_element(driver, By.CSS_SELECTOR, '.view-lines')
    code_block_text = code_block_element.text    

    solution = kadabra.main(f"I would like you to solve a problem in {language}, please only return code, do not include any additional text and do not add comments within the code, do not use any shortcuts for brevity (write the entire solution) and do not add ```cpp or ``` at the beginning or end of the solution. Here is the description of the problem: {description_text} and use this format to provide the solution {code_block_text}")

    charstoclear = len(code_block_element.text)

    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    textwindow.click()

    #press down arrow the length of code block text in case text.window.click() is not at the bottom (to start clearing)
    for _ in range(len(code_block_text.split('\n'))+7):
        textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ARROW_DOWN).perform()

    #press backspace double the amount of characters in the string of code_block_text to clear the window
    for _ in range(charstoclear + charstoclear):
        textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.BACKSPACE).perform()

    litch = solution.split('\n')

    for line in litch:
        ActionChains(driver).move_to_element(textwindow).send_keys(line).perform()
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ENTER).perform()
    
    



In [ ]:
#have to click run before submit

In [6]:
def click_submit(driver):
    #click submit 
    submit = wait_and_find_element(driver,By.CSS_SELECTOR, "button[data-e2e-locator='console-submit-button']")
    submit.click()
# click_submit(driver)

In [198]:
click_next_button(driver)

In [212]:
time.sleep(3)
handle_subscribe_popup(driver)

In [204]:
time.sleep(3)
solution_input(driver,'C++')

In [7]:
def format(driver):
    time.sleep(1)
    thiscontainer = WebDriverWait(driver, 1).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "flex.h-8.items-center.justify-between.border-b.p-1.border-border-quaternary"))
    )
    # click format code button
    button = thiscontainer.find_element(By.CSS_SELECTOR, "button[data-state='closed']")
    button.click()
    
    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    textwindow.click()
    # "scroll" down to the bottom
    for _ in range(len(solution.split('\n')) + 25):
        textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ARROW_DOWN).perform()

    #remove the extra closing brackets in the formatted code
    extras = solution.count('}')
    for _ in range((extras * 2) - 2):
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.BACKSPACE).perform()




In [205]:
format(driver)

'<selenium.webdriver.firefox.webdriver.WebDriver (session="911c548c-fa06-4b96-8712-4ae11977f0fa")>'

In [8]:
def click_format_button(driver):
    time.sleep(1)
    thiscontainer = WebDriverWait(driver, 1).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "flex.h-8.items-center.justify-between.border-b.p-1.border-border-quaternary"))
    )
    # click format code button
    button = thiscontainer.find_element(By.CSS_SELECTOR, "button[data-state='closed']")
    button.click()

In [9]:
def scrolldown(driver):   
    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
    textwindow.click()
    # "scroll" down to the bottom
    for _ in range(len(solution.split('\n')) + 25):
        textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.ARROW_DOWN).perform()

In [10]:
def remove_brackets(driver):
    textwindow = wait_and_find_element(driver, By.CLASS_NAME, "view-lines.monaco-mouse-cursor-text")    
    extras = solution.count('}')
    for _ in range((extras * 2) - 1):
        ActionChains(driver).move_to_element(textwindow).send_keys(Keys.BACKSPACE).perform()

In [12]:


while True:

    click_next_button(driver)
    handle_subscribe_popup(driver)
    solution_input(driver,'C++')
    click_format_button(driver)
    scrolldown(driver)
    remove_brackets(driver)
    click_submit(driver)

KeyboardInterrupt: 

In [258]:


(-2147483648 % 4)

0